In [0]:
import mlflow
from mlflow.deployments import get_deploy_client
client = get_deploy_client("databricks")


In [0]:
response = client.predict(
    endpoint="ka-fce5e7c5-endpoint",
    inputs={
        "messages": [{"role": "user", "content": 'testing battery'}],
        "databricks_options": {"return_trace": True}
    }
)
response

In [0]:
import os
import requests
import numpy as np
import pandas as pd
import json

def create_tf_serving_json(data):
    return {'inputs': {name: data[name].tolist() for name in data.keys()} if isinstance(data, dict) else data.tolist()}

def score_model(dataset):
    url = 'https://db-dais-2025.cloud.databricks.com/serving-endpoints/agents_telecommunications-self_healing_networks-vendor_manuals/invocations'
    headers = {'Authorization': f'Bearer {os.environ.get("DATABRICKS_TOKEN")}', 'Content-Type': 'application/json'}
    ds_dict = {'dataframe_split': dataset.to_dict(orient='split')} if isinstance(dataset, pd.DataFrame) else create_tf_serving_json(dataset)
    data_json = json.dumps(ds_dict, allow_nan=True)
    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    return response.json()

In [0]:
testdf = pd.DataFrame({'request': ['testing123'], 'id': [1]})

score_model(testdf)

In [0]:
import mlflow.deployments
from yaml import safe_load
from pyspark.sql.functions import concat, lit, col

with open("./params.yml", "r") as f:
    params = safe_load(f)

CATALOG = params.get('data_params').get('catalog')
SCHEMA = params.get('data_params').get('schema')
df = spark.table(f"{CATALOG}.{SCHEMA}.device_status_triage").withColumn('query_column', concat(col('device'), lit(' message: '), col('telemetry_error') ))

def make_prompt(device_error):
    return f"""Please find an automated and remote fix for this device and error message: {device_error}. Be as concise as possible, preferably using fewer than 20 words. If such a solution exists without requiring human intervention, start the response with the phrase AUTOMATED FIX followed by a 1-sentence overview. If human intervention is necessary, start the response with the phrase HUMAN INTERVENTION REQUIRED and state a single phrase overview for the course of action needed.
    """

errorslist = df.select('query_column').toPandas().values.flatten().tolist()
errorslist_solution = [make_prompt(x) for x in errorslist]


client = mlflow.deployments.get_deploy_client("databricks")
response = client.predict(
    endpoint="agents_telecommunications-self_healing_networks-vendor_manuals",
    inputs={
        "messages": [{"role": "user", "content": x} for x in errorslist_solution],
        "temperature": 0.2,
        "n": 1,
        "max_tokens": 500,
        "databricks_options": {
            "return_trace": True,
        }
    }
)
print(response)

In [0]:
responses = response['predictions']

parsed_responses = []
for res in responses:
    content = res['content']
    if content.startswith("AUTOMATED FIX"):
        parsed_responses.append({"type": "automated", "message": content})
    elif content.startswith("HUMAN INTERVENTION REQUIRED"):
        parsed_responses.append({"type": "human", "message": content})
    else:
        parsed_responses.append({"type": "unknown", "message": content})

display(parsed_responses)

In [0]:
for i in range(len(response.get('messages'))):
    print(response.get('messages')[i].get('content'))

In [0]:
response.get('messages')[-1].get('content')

In [0]:
%sql
insert into telecommunications.self_healing_networks.device_status values (
'47d52e21-8173-4371-b5e6-4b39481d2225', 	'550495e3-0620-4ed0-9622-51b819bf5596',	'Nokia AirScale Baseband Unit', 	'Automated Fix',	'2025-06-05T19:58:34.637+00:00'),
('3672293f-d942-4701-a148-882f1047f9e5',	'0defdc16-9bf5-43c3-b58f-9a925cd0830d',	'Delta D750 DC Power System',	'Automated Fix',	'2025-06-05T19:58:34.637+00:00'
)